# 1. Introduction:
The West Nile Virus ("WNV") has been a significant issue plaguing the United States of America ("USA") since 1999. The Centers for Disease Control and Prevention ("CDC") has acknowledged WNV as one of the leading causes of mosquito-borne diseases in the continental USA. However, there are no vaccines nor medication to treat WNV and according to the CDC, 1 in 5 people who are infected will develop a fever and other accompanying symptoms whilst 1 in 150 people will develop a more serious, potentially fatal, illness.

In 2002, the first human cases of West Nile virus were reported in Chicago. By 2004 the City of Chicago and the Chicago Department of Public Health ("CDPH") had established a comprehensive surveillance and control program that is still in effect today.

Every week from late spring through the fall, mosquitos in traps across the city are tested for the virus. The results of these tests influence when and where the city will spray airborne pesticides to control adult mosquito populations.

With this in mind, **our project's goal is to predict outbreaks of the WNV.** This will help the CDPH and the City of Chicago to more efficiently and effectively allocate resources towards preventing transmissions of this potentially deadly virus. Specifically, we will be using a combination of weather, time and location features to predict the presence of WNV within various mosquito traps laid out throughout Chicago.

Our project will additionally determine **the best stratgegy for controlling the spread of the WNV**, as well as discussing and justifying the various trade-offs that need to be made through implementing our model.


## Problem Statement
- Predict outbreaks of the NSW virus. Provide recommendations to conduct spraying operations. 

# 2. Importing the Libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 3. Data Import

In [2]:
# Load datasets
train = pd.read_csv('./datasets/train.csv')
test = pd.read_csv('./datasets/test.csv')
weather = pd.read_csv('./datasets/weather.csv')
spray = pd.read_csv('./datasets/spray.csv')

# 4. Weather - Data Cleaning

## 4.1. Weather - Initial Analysis

In [3]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [4]:
# Checking for missing values
weather.isnull().sum()

Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
Depth          0
Water1         0
SnowFall       0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

In [5]:
# Checking for duplicate values
weather.duplicated().sum()

0

In [6]:
# Many 'M' in values. According to documentation M = 'Missing Data'
weather.isin(['M']).sum().sort_values(ascending=False)

Water1         2944
Depart         1472
SnowFall       1472
Depth          1472
Tavg             11
Heat             11
Cool             11
SeaLevel          9
WetBulb           4
StnPressure       4
AvgSpeed          3
PrecipTotal       2
ResultDir         0
ResultSpeed       0
Station           0
CodeSum           0
Date              0
Sunrise           0
DewPoint          0
Tmin              0
Tmax              0
Sunset            0
dtype: int64

In [7]:
# Some 'T' values. According to documentation T = 'Trace'
weather.isin(['  T']).sum()

Station          0
Date             0
Tmax             0
Tmin             0
Tavg             0
Depart           0
DewPoint         0
WetBulb          0
Heat             0
Cool             0
Sunrise          0
Sunset           0
CodeSum          0
Depth            0
Water1           0
SnowFall        12
PrecipTotal    318
StnPressure      0
SeaLevel         0
ResultSpeed      0
ResultDir        0
AvgSpeed         0
dtype: int64

In [8]:
# Checking for zero values
weather.isin(['0']).sum()

Station           0
Date              0
Tmax              0
Tmin              0
Tavg              0
Depart            0
DewPoint          0
WetBulb           0
Heat           1870
Cool              0
Sunrise           0
Sunset            0
CodeSum           0
Depth          1472
Water1            0
SnowFall          0
PrecipTotal       0
StnPressure       0
SeaLevel          0
ResultSpeed       0
ResultDir         0
AvgSpeed          0
dtype: int64

In [9]:
# Checking for zero values. 
weather.isin(['0.0']).sum()

Station           0
Date              0
Tmax              0
Tmin              0
Tavg              0
Depart            0
DewPoint          0
WetBulb           0
Heat              0
Cool              0
Sunrise           0
Sunset            0
CodeSum           0
Depth             0
Water1            0
SnowFall       1459
PrecipTotal       0
StnPressure       0
SeaLevel          0
ResultSpeed       0
ResultDir         0
AvgSpeed          0
dtype: int64

In [10]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Station      2944 non-null   int64  
 1   Date         2944 non-null   object 
 2   Tmax         2944 non-null   int64  
 3   Tmin         2944 non-null   int64  
 4   Tavg         2944 non-null   object 
 5   Depart       2944 non-null   object 
 6   DewPoint     2944 non-null   int64  
 7   WetBulb      2944 non-null   object 
 8   Heat         2944 non-null   object 
 9   Cool         2944 non-null   object 
 10  Sunrise      2944 non-null   object 
 11  Sunset       2944 non-null   object 
 12  CodeSum      2944 non-null   object 
 13  Depth        2944 non-null   object 
 14  Water1       2944 non-null   object 
 15  SnowFall     2944 non-null   object 
 16  PrecipTotal  2944 non-null   object 
 17  StnPressure  2944 non-null   object 
 18  SeaLevel     2944 non-null   object 
 19  Result

## 4.2. Weather - Imputing / Dropping Features

In [11]:
# Dropping Water1, Depth and SnowFall as their data is most likely not imputable and therefore not of use to us.
weather.drop(columns=['Water1', 'Depth', 'SnowFall'], inplace=True)

In [12]:
# Function to impute Average Temperature
def impute_tavg(row):
    # Impute missing values for Tavg
    if row['Tavg'] == 'M':
        row['Tavg'] = round((row['Tmax'] + row['Tmin']) / 2)
     
    return row

In [13]:
weather = weather.apply(impute_tavg, axis=1)

In [14]:
# Tavg is no longer 'M'
weather.isin(['M']).sum().sort_values(ascending=False)

Depart         1472
Cool             11
Heat             11
SeaLevel          9
WetBulb           4
StnPressure       4
AvgSpeed          3
PrecipTotal       2
DewPoint          0
Tavg              0
Date              0
Sunrise           0
Sunset            0
CodeSum           0
Tmin              0
Tmax              0
ResultSpeed       0
ResultDir         0
Station           0
dtype: int64

In [15]:
# Function to impute Cooling and Heating Degrees - Base is 65

def impute_heat_cool(row):
    # Impute missing values for Cool and Warm
    if row['Cool'] == 'M' or row['Heat'] == 'M':
        heat_difference = 65 - row['Tavg']
        if heat_difference < 0: 
            row['Cool'] = heat_difference
            row['Heat'] = 0
        elif heat_difference > 0:
            row['Cool'] = 0
            row['Heat'] = heat_difference
        else:
            row['Cool'] = row['Heat'] = 0
    return row

In [16]:
weather = weather.apply(impute_heat_cool, axis=1)

In [17]:
# Heat/Cool is no longer 'M'
weather.isin(['M']).sum().sort_values(ascending=False)

Depart         1472
SeaLevel          9
StnPressure       4
WetBulb           4
AvgSpeed          3
PrecipTotal       2
Sunset            0
ResultDir         0
ResultSpeed       0
CodeSum           0
Station           0
Sunrise           0
Date              0
Heat              0
DewPoint          0
Tavg              0
Tmin              0
Tmax              0
Cool              0
dtype: int64

In [18]:
# Impute station 2 with departure from station 1's 30 year normal
def impute_depart(row):
    if row['Station'] == 2:
        # Difference between avg temp of two stations
        difference = int(row['Tavg']) - int(weather.loc[
            (weather['Date'] == row['Date']) & 
            (weather['Station'] == 1), 
            'Tavg'
        ])
        # Impute with station 1's readings plus difference
        row['Depart'] = int(weather.loc[
            (weather['Date'] == row['Date']) & 
            (weather['Station'] == 1), 
            'Depart'
        ]) + difference
    return row

In [19]:
weather = weather.apply(impute_depart, axis=1)

In [20]:
# Depart is no longer 'M'
weather.isin(['M']).sum().sort_values(ascending=False)

SeaLevel       9
StnPressure    4
WetBulb        4
AvgSpeed       3
PrecipTotal    2
Sunrise        0
ResultDir      0
ResultSpeed    0
CodeSum        0
Sunset         0
Station        0
Date           0
Heat           0
DewPoint       0
Depart         0
Tavg           0
Tmin           0
Tmax           0
Cool           0
dtype: int64

In [21]:
weather[weather['StnPressure'] == 'M']

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
87,2,2007-06-13,86,68,77,9,53,62,0,12,-,-,,0.00,M,M,7.0,5,M
848,1,2009-06-26,86,69,78,7,60,M,0,13,0418,1931,,0.00,M,29.85,6.4,4,8.2
2410,1,2013-08-10,81,64,73,0,57,M,0,8,0454,1900,,0.00,M,30.08,5.3,5,6.5
2411,2,2013-08-10,81,68,75,2,55,63,0,10,-,-,,0.00,M,30.07,6.0,6,7.4


In [22]:
# Impute StnPressure by interpolating from previous & next day values
for index, row in weather.loc[weather['StnPressure'].isin(['M'])].iterrows():
    inter = (float(weather.iloc[(index - 2)]['StnPressure']) + \
             float(weather.iloc[(index + 2)]['StnPressure'])) / 2
    weather.at[index, 'StnPressure'] = round(inter, 2)

In [23]:
# StnPressure is no longer 'M'
weather.isin(['M']).sum().sort_values(ascending=False)

SeaLevel       9
WetBulb        4
AvgSpeed       3
PrecipTotal    2
Sunrise        0
ResultDir      0
ResultSpeed    0
StnPressure    0
CodeSum        0
Sunset         0
Station        0
Date           0
Heat           0
DewPoint       0
Depart         0
Tavg           0
Tmin           0
Tmax           0
Cool           0
dtype: int64

In [24]:
# Impute SeaLevel by interpolating from previous & next day values
for index, row in weather.loc[weather['SeaLevel'].isin(['M'])].iterrows():
    inter = (float(weather.iloc[(index - 2)]['SeaLevel']) + \
             float(weather.iloc[(index + 2)]['SeaLevel'])) / 2
    weather.at[index, 'SeaLevel'] = round(inter, 2)

In [25]:
# SeaLevel is no longer 'M'
weather.isin(['M']).sum().sort_values(ascending=False)

WetBulb        4
AvgSpeed       3
PrecipTotal    2
Sunrise        0
ResultDir      0
ResultSpeed    0
SeaLevel       0
StnPressure    0
CodeSum        0
Sunset         0
Station        0
Date           0
Heat           0
DewPoint       0
Depart         0
Tavg           0
Tmin           0
Tmax           0
Cool           0
dtype: int64

In [26]:
# Imputing remaining missing values being wetbulb, average speed, preciptotal
def impute_remaining(row):
    if row['WetBulb'] == 'M':
        if row['Station'] == 1:
            row['WetBulb'] = weather.loc[
                (weather['Date'] == row['Date']) & 
                (weather['Station'] == 2), 
                'WetBulb'
            ].values[0]
        else:
            row['WetBulb'] = weather.loc[
                (weather['Date'] == row['Date']) & 
                (weather['Station'] == 1), 
                'WetBulb'
            ].values[0]
        
    if row['AvgSpeed'] == 'M':
        row['AvgSpeed'] = weather.loc[
            (weather['Date'] == row['Date']) & 
            (weather['Station'] == 1), 
            'AvgSpeed'
        ].values[0]
        
    if row['PrecipTotal'] == 'M':
        row['PrecipTotal'] = weather.loc[
            (weather['Date'] == row['Date']) & 
            (weather['Station'] == 1), 
            'PrecipTotal'
        ].values[0]
    
    return row

In [27]:
weather = weather.apply(impute_remaining, axis=1)

In [28]:
# No more missing values.
weather.isin(['M']).sum()

Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

In [29]:
# Earlier on we still had some values that were "Trace" values, and we also have Missing values being '-'
weather.isin(['-', '  T']).any()

Station        False
Date           False
Tmax           False
Tmin           False
Tavg           False
Depart         False
DewPoint       False
WetBulb        False
Heat           False
Cool           False
Sunrise         True
Sunset          True
CodeSum        False
PrecipTotal     True
StnPressure    False
SeaLevel       False
ResultSpeed    False
ResultDir      False
AvgSpeed       False
dtype: bool

In [30]:
# Changing PrecipTotal trace to 0.01
weather['PrecipTotal'] = weather['PrecipTotal'].apply(lambda x: 0.01 if 'T' in x else x)

In [31]:
# Change Date column from string to datetime
weather['Date'] = pd.to_datetime(weather['Date'])

In [32]:
# Impute sunrise/sunset
def impute_sun(row):
    if row['Station'] == 2:
        row['Sunrise'] = weather[(weather['Date'] == row['Date']) & (weather['Station'] == 1)]['Sunrise'].values[0]
        row['Sunset'] = weather[(weather['Date'] == row['Date']) & (weather['Station'] == 1)]['Sunset'].values[0]
    return row

In [33]:
weather = weather.apply(impute_sun, axis=1)

In [34]:
# We no longer have any missing '-' or T values
weather.isin(['-', '  T']).any()

Station        False
Date           False
Tmax           False
Tmin           False
Tavg           False
Depart         False
DewPoint       False
WetBulb        False
Heat           False
Cool           False
Sunrise        False
Sunset         False
CodeSum        False
PrecipTotal    False
StnPressure    False
SeaLevel       False
ResultSpeed    False
ResultDir      False
AvgSpeed       False
dtype: bool

## 4.3. Weather - Addressing CodeSum Column

In [35]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,15,51,57,0,3,0448,1849,,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,-2,42,47,13,0,0447,1850,BR HZ,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0.00,29.39,30.12,11.7,7,11.9


In [36]:
snow = ['SN', 'SG', 'GS', 'PL', 'IC', 'DR', 'BC']
windy = ['SQ', 'DS', 'SS', 'PO', 'BL']
rain = ['TS', 'GR', 'RA', 'DZ', 'SH']
lowvis = ['FG+', 'FG', 'BR', 'HZ']
codesum_others = ['UP', 'VA', 'DU', 'SA', 'FZ']

In [37]:
def codesum_split(row):
    codes = row['CodeSum'].split()
    
    # Check for weather conditions
    if any(code in codes for code in snow):
        row['snow'] = 1
    if any(code in codes for code in windy):
        row['windy'] = 1 
    if any(code in codes for code in rain):
        row['rain'] = 1 
    if any(code in codes for code in lowvis):
        row['lowvis'] = 1
    if any(code in codes for code in codesum_others):
        row['codesum_others'] = 1
        
    return row

In [38]:
weather = weather.apply(codesum_split, axis=1)

In [39]:
weather[['snow', 'windy', 'rain', 'lowvis']].sum()

snow         6.0
windy        3.0
rain      1007.0
lowvis     882.0
dtype: float64

In [40]:
weather = weather.fillna(0)

In [41]:
weather.isnull().sum()

AvgSpeed       0
CodeSum        0
Cool           0
Date           0
Depart         0
DewPoint       0
Heat           0
PrecipTotal    0
ResultDir      0
ResultSpeed    0
SeaLevel       0
Station        0
StnPressure    0
Sunrise        0
Sunset         0
Tavg           0
Tmax           0
Tmin           0
WetBulb        0
lowvis         0
rain           0
snow           0
windy          0
dtype: int64

In [42]:
# Dropping snow & windy due to extremely low variance
weather = weather.drop(columns=['snow', 'windy'])

In [43]:
# Convert remaining columns
for col in weather.columns:
    try:
        weather[col] = weather[col].astype(float)
    except:
        print(col, 'cannot be transformed into float')
        pass

CodeSum cannot be transformed into float
Date cannot be transformed into float


## 4.4. Merging Station 1 and Station 2 Values

In [44]:
weather = weather.groupby('Date').sum() / 2
weather = weather.drop(columns=['Station']).reset_index()

In [45]:
weather

,Date,AvgSpeed,Cool,Depart,DewPoint,Heat,PrecipTotal,ResultDir,ResultSpeed,SeaLevel,StnPressure,Sunrise,Sunset,Tavg,Tmax,Tmin,WetBulb,lowvis,rain
0,2007-05-01,9.40,2.5,14.5,51.0,0.0,0.000,26.0,2.20,29.820,29.140,448.0,1849.0,67.5,83.5,51.0,56.5,0.0,0.0
1,2007-05-02,13.40,0.0,-2.5,42.0,13.5,0.000,3.0,13.15,30.085,29.410,447.0,1850.0,51.5,59.5,42.5,47.0,1.0,0.0
2,2007-05-03,12.55,0.0,3.0,40.0,8.0,0.000,6.5,12.30,30.120,29.425,446.0,1851.0,57.0,66.5,47.0,49.0,0.5,0.0
3,2007-05-04,10.60,0.0,7.0,41.5,4.0,0.005,7.5,10.25,30.045,29.335,444.0,1852.0,61.0,72.0,50.0,50.0,0.0,0.5
4,2007-05-05,11.75,0.0,5.0,38.5,5.0,0.010,7.0,11.45,30.095,29.430,443.0,1853.0,60.0,66.0,53.5,49.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,2014-10-27,13.25,1.0,17.5,51.5,0.5,0.010,19.0,12.35,29.665,28.960,618.0,1653.0,65.5,78.0,52.5,58.5,0.0,0.5
1468,2014-10-28,15.10,0.0,10.0,39.0,8.0,0.020,26.0,14.40,29.850,29.190,619.0,1651.0,57.0,67.0,46.5,47.5,0.0,0.5
1469,2014-10-29,9.45,0.0,-3.0,33.0,21.0,0.000,29.0,9.00,30.065,29.390,620.0,1650.0,44.0,49.0,38.0,41.0,0.0,0.0
1470,2014-10-30,6.00,0.0,-2.5,34.5,21.5,0.005,23.5,5.50,30.095,29.375,622.0,1649.0,43.5,52.0,34.5,41.0,0.0,0.5


In [46]:
weather.to_csv('./datasets/cleaned_weather.csv', index=False)

# 5. Train - Data Cleaning

## 5.1. Train - Initial Analysis

In [47]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [48]:
train.isnull().sum()

Date                      0
Address                   0
Species                   0
Block                     0
Street                    0
Trap                      0
AddressNumberAndStreet    0
Latitude                  0
Longitude                 0
AddressAccuracy           0
NumMosquitos              0
WnvPresent                0
dtype: int64

In [49]:
train.shape

(10506, 12)

In [50]:
train.columns

Index(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent'],
      dtype='object')

In [51]:
train.dtypes

Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
dtype: object

In [52]:
train.duplicated().sum()

813

In [53]:
train.drop_duplicates(ignore_index=True, inplace=True)

In [54]:
train['Date'] = pd.to_datetime(train['Date'])

In [55]:
train.to_csv('./datasets/cleaned_train.csv', index=False)

# 6. Test - Data Cleaning

## 6.1. Test - Initial Analysis

In [56]:
test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [57]:
test.isnull().sum()

Id                        0
Date                      0
Address                   0
Species                   0
Block                     0
Street                    0
Trap                      0
AddressNumberAndStreet    0
Latitude                  0
Longitude                 0
AddressAccuracy           0
dtype: int64

In [58]:
test.shape

(116293, 11)

In [59]:
test.columns

Index(['Id', 'Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy'],
      dtype='object')

In [60]:
test.dtypes

Id                          int64
Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
dtype: object

In [61]:
test.duplicated().sum()

0

In [62]:
test['Date'] = pd.to_datetime(test['Date'])

In [63]:
test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [64]:
test.to_csv('./datasets/test.csv', index=False)

# 7. Spray - Data Cleaning

## 7.1. Spray - Initial Analysis

In [65]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [66]:
spray.isnull().sum()

Date           0
Time         584
Latitude       0
Longitude      0
dtype: int64

In [67]:
spray.shape

(14835, 4)

In [68]:
spray.columns

Index(['Date', 'Time', 'Latitude', 'Longitude'], dtype='object')

In [69]:
spray.dtypes

Date          object
Time          object
Latitude     float64
Longitude    float64
dtype: object

In [70]:
spray.duplicated().sum()

541

In [71]:
# Not able to impute time values
spray.drop_duplicates(inplace=True)

In [72]:
spray.drop(columns='Time',inplace=True)

In [73]:
spray['Date'] = pd.to_datetime(spray['Date'])

In [74]:
spray.to_csv('./datasets/cleaned_spray.csv', index=False)